# Talkativ: 토픽 추천 시스템

## 개선 사항

| 기존 | 개선 |
|------|------|
| 수동 키워드 정의 (160개) | AI-Hub 데이터 기반 키워드 (500+개) |
| 단순 키워드 매칭 | TF-IDF 가중치 적용 |
| 16개 토픽 | 20개 토픽 (AI-Hub 기준) |

### 데이터 소스
- **AI-Hub 주제별 텍스트 일상 대화 데이터**
- URL: https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=543
- 총 10,644개 대화, 20개 토픽

## 전체 구조 요약

**사용자–아바타 대화**에서 “지금 상황에 가장 자연스럽고 안전한 대화 주제”를 고르고, 그 주제로 대화 시작 문장(Conversation Starter)까지 생성하는 **토픽 추천 기능**을 구현합니다.

### 입력(Inputs)
- **User Profile** (`PersonProfile`): 사용자의 관심사/회피 토픽/언어 수준 등 “개인화” 정보  
- **Avatar Profile** (`PersonProfile`): 교수/친구/낯선 사람 등 “관계(relationship)·말투” 결정에 필요한 정보  
- **Current Dialogue** (문자열 또는 최근 턴 요약): “현재 대화 흐름(맥락)” 정보

### 출력(Outputs)
- **Ranked Topics**: 점수 기반으로 정렬된 추천 토픽 목록  
- **Conversation Starters**: 각 토픽에 대해 바로 사용할 수 있는 자연스러운 시작 문장

### 4-Layer 추천 파이프라인
아래 파이프라인은 “안전성 → 관계 적합성 → 대화 흐름” 순으로 점진적으로 정교해지는 구조입니다.

```
Input: User Profile + Avatar Profile + Current Dialogue
                    ↓
Layer 1: Rule-Based Filtering      (민감/회피 토픽 제거, 공통 관심사 우선)
                    ↓
Layer 2: Relationship Scoring      (관계/지위/친밀도 기반 적합성 점수)
                    ↓
Layer 3: Context-Aware Ranking     (현재 대화 토픽/전환 자연스러움 반영)
                    ↓
Layer 4: Starter Generation        (템플릿 우선 + 선택적 LLM + 캐시)
                    ↓
Output: Ranked Topics + Conversation Starters
```

## 중요한 설계 철학
1) **필터링을 먼저** 해서 “추천하면 안 되는 것”을 제거  
2) 그 다음 “관계에 맞는가?”를 평가  
3) 마지막에 “지금 흐름에서 자연스러운가?”를 반영  
4) 시작 문장은 **템플릿으로 안정성 확보** 후, 필요할 때만 LLM을 사용하여 비용/불안정성을 줄입니다.


---
## 1. 환경 설정

### 1-1. 환경 설정 코드 설명 (Imports & Mode)

이 섹션의 코드는 노트북 전체에서 쓰이는 라이브러리와 실행 모드를 준비합니다.

####  주요 라이브러리 역할
- `re, math` : 텍스트 전처리(정규식), 점수 계산
- `dataclasses` : `PersonProfile`, `TopicRecommendation` 같은 구조체를 간결하게 정의
- `collections.Counter` : 키워드 빈도 계산(토픽 탐지 점수에 사용)
- `ipywidgets` (선택) : 데모 환경에서 모드/파라미터를 토글할 때 사용 가능

####  `USE_LLM` 플래그의 의미
- `USE_LLM = False` : **시뮬레이션 모드**  
  - 토픽 추천과 문장 생성이 *완전히 규칙/템플릿 기반*으로 동작
  - 데모 안정성 높고 비용 0
- `USE_LLM = True` : **LLM 모드(확장 가능)**  
  - Layer 4에서 “상위 N개 토픽”만 LLM로 더 자연스럽게 다듬는 방식으로 확장 가능
  - 이 노트북은 **LLM 호출부를 ‘후킹(hook)’ 형태**로 둬서 나중에 API를 연결하기 쉽습니다.


In [1]:
import os
import re
import json
import math
import requests
import random
from typing import List, Dict, Set, Tuple, Optional
from dataclasses import dataclass, field
from collections import Counter, defaultdict
from enum import Enum

print(" 라이브러리 로드 완료")

 라이브러리 로드 완료


In [2]:
# 모드 설정
USE_LLM = False

try:
    import ipywidgets as widgets
    from IPython.display import display

    mode_toggle = widgets.ToggleButtons(
        options=[("🔧 Simulation", False), (" LLM (CLOVA Studio)", True)],
        value=USE_LLM,
        description="Mode:"
    )

    def on_mode_change(change):
        global USE_LLM
        USE_LLM = change['new']
        print(f"모드: {'LLM' if USE_LLM else 'Simulation'}")

    mode_toggle.observe(on_mode_change, names='value')
    display(mode_toggle)
except:
    pass

print(f"현재 모드: {'LLM' if USE_LLM else 'Simulation'}")

ToggleButtons(description='Mode:', options=(('🔧 Simulation', False), (' LLM (CLOVA Studio)', True)), value=Fal…

현재 모드: Simulation


---
## 2. AI-Hub 데이터 기반 토픽 분류 체계

AI-Hub 데이터에서 추출한 실제 키워드를 사용합니다.

### 2-1. AI-Hub 토픽 체계 데이터 구조 설명

이 섹션은 AI-Hub의 “20개 토픽”을 코드에서 사용하기 위한 **정규화된 토픽 사전**입니다.  
(**데이터셋의 라벨 체계를 그대로 반영**)

#### `AI_HUB_TOPICS`에 들어있는 정보(예시)
각 토픽은 보통 아래 필드를 가집니다.
- `name_ko` / `name_en` : 토픽의 한/영 이름
- `keywords` : 해당 토픽을 대표하는 키워드 목록 (AI-Hub 대화에서 추출/정리)
- (추가 가능) `sensitive` / `avoid` 같은 플래그: 안전 필터링에 활용

#### 왜 “키워드 사전”이 필요한가?
Layer 3(맥락 기반 랭킹)에서 **현재 대화가 어떤 토픽을 다루는지 탐지**해야 합니다.  
이를 위해 “대화 텍스트 ↔ 토픽 키워드”의 연결이 필요합니다.

#### 확장 포인트
- 토픽을 추가하려면:
  1) `AI_HUB_TOPICS`에 새 토픽 항목 추가  
  2) `STARTER_TEMPLATES`에 해당 토픽 시작 문장 템플릿 추가  
  3) (선택) 전환 그래프에 자연스러운 다음 토픽 연결 추가


In [3]:
# ============================================================
# AI-Hub 기반 토픽 분류 체계 (20개 토픽)
# 키워드는 실제 대화 데이터에서 추출
# ============================================================

TOPIC_TAXONOMY = {
    # ========== 가족/관계 ==========
    "가족": {
        "name_ko": "가족",
        "name_en": "Family",
        "category": "relationship",
        "keywords": [
            "엄마", "아빠", "부모님", "동생", "언니", "오빠", "누나", "형",
            "가족", "가족들", "어머니", "아버지", "할머니", "할아버지",
            "조카", "삼촌", "이모", "고모", "외삼촌", "사촌",
            "집안", "명절", "추석", "설날", "제사", "가족여행"
        ],
        "weight": {"엄마": 2.0, "아빠": 2.0, "가족": 2.5, "부모님": 2.0},
        "formality_min": "polite",
        "is_sensitive": False
    },

    "연애/결혼": {
        "name_ko": "연애/결혼",
        "name_en": "Dating & Marriage",
        "category": "relationship",
        "keywords": [
            "결혼", "연애", "남자친구", "여자친구", "남편", "와이프", "아내",
            "결혼식", "신혼", "예비신부", "예비신랑", "청첩장",
            "기념일", "데이트", "소개팅", "선물", "커플",
            "사귀다", "헤어지다", "이별", "고백", "프로포즈", "육아"
        ],
        "weight": {"결혼": 2.5, "연애": 2.5, "남자친구": 2.0, "여자친구": 2.0},
        "formality_min": "informal",
        "is_sensitive": True
    },

    # ========== 일/학업 ==========
    "회사/아르바이트": {
        "name_ko": "회사/아르바이트",
        "name_en": "Work & Part-time Job",
        "category": "work",
        "keywords": [
            "회사", "알바", "아르바이트", "직장", "취업", "이직",
            "출근", "퇴근", "야근", "회식", "업무", "월급", "시급",
            "상사", "팀장", "부장", "사장", "동료", "인턴",
            "면접", "이력서", "자소서", "퇴사", "휴가"
        ],
        "weight": {"회사": 2.5, "알바": 2.5, "취업": 2.0, "회식": 1.5},
        "formality_min": "polite",
        "is_sensitive": False
    },

    "교육": {
        "name_ko": "교육",
        "name_en": "Education",
        "category": "academic",
        "keywords": [
            "공부", "학원", "학교", "수업", "시험", "선생님", "교수님",
            "영어", "수학", "과학", "국어", "토익", "토플",
            "자격증", "수능", "대학", "입시", "과외",
            "성적", "학점", "중간고사", "기말고사", "레포트", "과제",
            "동아리", "MT", "OT", "수강신청", "졸업"
        ],
        "weight": {"공부": 2.0, "학원": 2.0, "시험": 2.0, "학교": 1.5},
        "formality_min": "polite",
        "is_sensitive": False
    },

    "군대": {
        "name_ko": "군대",
        "name_en": "Military",
        "category": "life",
        "keywords": [
            "군대", "군인", "훈련", "휴가", "전역", "입대", "복무",
            "육군", "해군", "공군", "해병대", "부대", "보직",
            "이등병", "일병", "상병", "병장", "장교", "부사관",
            "훈련소", "자대", "군생활", "면회", "군복"
        ],
        "weight": {"군대": 3.0, "군인": 2.0, "전역": 2.0, "휴가": 1.5},
        "formality_min": "polite",
        "is_sensitive": False
    },

    # ========== 일상생활 ==========
    "주거와생활": {
        "name_ko": "주거와생활",
        "name_en": "Housing & Living",
        "category": "daily",
        "keywords": [
            "아파트", "주택", "빌라", "오피스텔", "원룸",
            "전세", "월세", "집값", "부동산", "매매", "청약", "대출",
            "이사", "인테리어", "리모델링", "자취", "동네",
            "청소", "빨래", "가구", "가전", "냉장고", "세탁기"
        ],
        "weight": {"아파트": 2.0, "전세": 2.0, "월세": 2.0, "이사": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "교통": {
        "name_ko": "교통",
        "name_en": "Transportation",
        "category": "daily",
        "keywords": [
            "버스", "지하철", "택시", "기차", "KTX", "비행기",
            "운전", "면허", "자동차", "차", "전기차", "테슬라",
            "대중교통", "출퇴근", "주차", "주차장",
            "사고", "교통사고", "신호등", "도로", "고속도로",
            "자전거", "킥보드", "오토바이"
        ],
        "weight": {"버스": 2.0, "지하철": 2.0, "운전": 1.5, "교통": 2.0},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "건강": {
        "name_ko": "건강",
        "name_en": "Health",
        "category": "daily",
        "keywords": [
            "건강", "운동", "병원", "의사", "약", "치료", "수술",
            "다이어트", "살", "체중", "헬스", "PT",
            "허리", "목", "어깨", "두통", "감기", "열",
            "영양제", "비타민", "보험", "건강검진",
            "정신건강", "스트레스", "우울", "불면증"
        ],
        "weight": {"건강": 2.5, "병원": 2.0, "운동": 1.5, "다이어트": 1.5},
        "formality_min": "polite",
        "is_sensitive": False
    },

    "미용": {
        "name_ko": "미용",
        "name_en": "Beauty",
        "category": "daily",
        "keywords": [
            "머리", "헤어", "미용실", "염색", "파마", "커트",
            "피부", "피부과", "여드름", "잡티", "주름",
            "화장", "메이크업", "화장품", "스킨케어",
            "눈썹", "속눈썹", "네일", "탈모", "두피",
            "성형", "시술", "레이저", "필러", "보톡스"
        ],
        "weight": {"머리": 1.5, "피부": 2.0, "미용실": 2.0, "화장": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "계절/날씨": {
        "name_ko": "계절/날씨",
        "name_en": "Weather & Season",
        "category": "daily",
        "keywords": [
            "날씨", "오늘날씨", "비", "눈", "우산", "태풍", "장마",
            "봄", "여름", "가을", "겨울", "계절",
            "더워", "추워", "덥다", "춥다", "시원", "따뜻",
            "에어컨", "난방", "히터", "선풍기",
            "단풍", "벚꽃", "눈사람", "패딩", "반팔"
        ],
        "weight": {"날씨": 3.0, "겨울": 2.0, "여름": 2.0, "비": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    # ========== 음식/쇼핑 ==========
    "식음료": {
        "name_ko": "식음료",
        "name_en": "Food & Drink",
        "category": "culture",
        "keywords": [
            "밥", "점심", "저녁", "아침", "식사", "맛집",
            "커피", "카페", "디저트", "케이크", "빵",
            "치킨", "피자", "햄버거", "라면", "떡볶이", "삼겹살",
            "배달", "배달음식", "야식", "술", "맥주", "소주",
            "요리", "레시피", "메뉴", "음식", "음료"
        ],
        "weight": {"맛집": 2.0, "커피": 1.5, "배달": 1.5, "치킨": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "상거래전반": {
        "name_ko": "쇼핑/상거래",
        "name_en": "Shopping & Commerce",
        "category": "daily",
        "keywords": [
            "쇼핑", "쇼핑몰", "온라인쇼핑", "인터넷쇼핑",
            "쿠팡", "네이버쇼핑", "11번가", "위메프", "티몬",
            "할인", "세일", "쿠폰", "적립금", "카드",
            "배송", "택배", "반품", "교환", "환불",
            "구매", "주문", "결제", "가격", "당근마켓", "중고"
        ],
        "weight": {"쇼핑": 2.0, "쿠팡": 2.0, "할인": 1.5, "배송": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    # ========== 취미/엔터테인먼트 ==========
    "게임": {
        "name_ko": "게임",
        "name_en": "Gaming",
        "category": "entertainment",
        "keywords": [
            "게임", "롤", "LOL", "배그", "오버워치", "피파",
            "닌텐도", "스위치", "플스", "플레이스테이션", "엑스박스",
            "모바일게임", "PC게임", "온라인게임",
            "캐릭터", "레벨", "아이템", "현질", "과금",
            "스팀", "e스포츠", "프로게이머", "오락실"
        ],
        "weight": {"게임": 3.0, "롤": 2.0, "닌텐도": 2.0, "배그": 2.0},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "스포츠/레저": {
        "name_ko": "스포츠/레저",
        "name_en": "Sports & Leisure",
        "category": "entertainment",
        "keywords": [
            "운동", "축구", "야구", "농구", "배구", "테니스", "골프",
            "헬스", "수영", "등산", "자전거", "러닝", "마라톤",
            "캠핑", "낚시", "볼링", "당구", "탁구",
            "올림픽", "월드컵", "경기", "선수", "팀",
            "스키", "스노보드", "서핑", "요가", "필라테스"
        ],
        "weight": {"축구": 2.0, "야구": 2.0, "캠핑": 2.0, "등산": 2.0},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "방송/연예": {
        "name_ko": "방송/연예",
        "name_en": "Broadcasting & Entertainment",
        "category": "entertainment",
        "keywords": [
            "드라마", "예능", "방송", "TV", "티비",
            "유튜브", "유튜버", "인플루언서", "스트리머",
            "아이돌", "가수", "연예인", "배우", "개그맨",
            "노래", "음악", "앨범", "콘서트", "팬미팅",
            "BTS", "블랙핑크", "투바투", "아이브"
        ],
        "weight": {"드라마": 2.0, "예능": 2.0, "아이돌": 2.0, "유튜브": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "영화/만화": {
        "name_ko": "영화/만화",
        "name_en": "Movies & Comics",
        "category": "entertainment",
        "keywords": [
            "영화", "영화관", "CGV", "메가박스", "롯데시네마",
            "넷플릭스", "왓챠", "디즈니플러스", "웨이브",
            "만화", "웹툰", "애니메이션", "애니",
            "마블", "DC", "디즈니", "픽사", "지브리",
            "감독", "배우", "개봉", "시사회", "스포일러"
        ],
        "weight": {"영화": 3.0, "넷플릭스": 2.0, "웹툰": 2.0, "만화": 2.0},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "여행": {
        "name_ko": "여행",
        "name_en": "Travel",
        "category": "entertainment",
        "keywords": [
            "여행", "해외여행", "국내여행", "휴가",
            "제주도", "부산", "강릉", "경주", "속초", "전주",
            "일본", "태국", "베트남", "유럽", "미국",
            "호텔", "숙소", "에어비앤비", "펜션", "리조트",
            "비행기", "공항", "여권", "비자", "환전"
        ],
        "weight": {"여행": 3.0, "제주도": 2.0, "해외여행": 2.0, "호텔": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    "반려동물": {
        "name_ko": "반려동물",
        "name_en": "Pets",
        "category": "daily",
        "keywords": [
            "강아지", "고양이", "반려동물", "펫", "애완동물",
            "산책", "간식", "사료", "목줄", "배변패드",
            "햄스터", "토끼", "물고기", "거북이", "앵무새",
            "동물병원", "예방접종", "중성화", "입양", "분양"
        ],
        "weight": {"강아지": 3.0, "고양이": 3.0, "반려동물": 2.5, "산책": 1.5},
        "formality_min": "informal",
        "is_sensitive": False
    },

    # ========== 민감 토픽 ==========
    "사회이슈": {
        "name_ko": "사회이슈",
        "name_en": "Social Issues",
        "category": "sensitive",
        "keywords": [
            "뉴스", "사건", "사고", "이슈", "기사",
            "코로나", "백신", "마스크", "확진자", "거리두기",
            "범죄", "사기", "피해", "경찰", "검찰",
            "정치", "선거", "대통령", "국회", "정당",
            "주식", "부동산", "물가", "경제"
        ],
        "weight": {"뉴스": 1.5, "코로나": 2.0, "사건": 1.5},
        "formality_min": "polite",
        "is_sensitive": True
    },

    "타국가이슈": {
        "name_ko": "타국가이슈",
        "name_en": "International Issues",
        "category": "sensitive",
        "keywords": [
            "미국", "중국", "일본", "북한", "러시아",
            "우리나라", "외국", "해외", "국제",
            "전쟁", "분쟁", "외교", "무역",
            "올림픽", "월드컵"
        ],
        "weight": {"미국": 1.5, "중국": 1.5, "일본": 1.5},
        "formality_min": "polite",
        "is_sensitive": True
    }
}

# 민감 토픽 목록
SENSITIVE_TOPICS = [tid for tid, data in TOPIC_TAXONOMY.items()
                    if data.get("is_sensitive", False)]

print(f" 토픽 분류 체계 정의 완료")
print(f"   - 총 {len(TOPIC_TAXONOMY)}개 토픽")
print(f"   - 민감 토픽: {SENSITIVE_TOPICS}")
print(f"   - 총 키워드 수: {sum(len(t['keywords']) for t in TOPIC_TAXONOMY.values())}개")

 토픽 분류 체계 정의 완료
   - 총 20개 토픽
   - 민감 토픽: ['연애/결혼', '사회이슈', '타국가이슈']
   - 총 키워드 수: 485개


---
## 3. 개선된 토픽 탐지기 (TF-IDF 가중치)

### 3-1. 개선된 토픽 탐지기(ImprovedTopicDetector) 상세 설명

이 섹션은 “현재 대화에서 어떤 토픽이 등장했는지”를 **점수화**합니다.  
단순히 “키워드가 있으면 해당 토픽”이 아니라, **중요 키워드는 더 크게 반영**하도록 설계되어 있습니다.

#### 입력/출력
- 입력: `text: str` (최근 대화 텍스트)
- 출력: `List[str]` (점수가 높은 토픽 이름 목록)

#### 핵심 아이디어: TF-IDF 가중치(가벼운 버전)
- 어떤 단어는 어느 토픽에서도 흔히 등장합니다. (예: “좋다”, “요즘”)
- 어떤 단어는 특정 토픽에서만 강하게 등장합니다. (예: “시험”, “면접”, “운동”)
- 그래서 **키워드별 중요도(weight)** 를 주고, 단순 빈도보다 “구분력 있는 단어”를 더 신뢰합니다.

#### 점수 계산 흐름(개념)
1) 텍스트 전처리: 소문자화/기호 제거 등
2) 단어 빈도 계산: `Counter`
3) 토픽별 점수 합산:
   - “토픽 키워드가 텍스트에 나타났는가?”
   - 나타났다면 그 키워드의 중요도(가중치)를 더함
4) 점수가 충분히 높으면 “현재 대화 토픽” 후보로 채택

#### 왜 Layer 3에서 중요?
Layer 3은 “지금 대화 흐름에서 자연스러운 주제 전환”을 만들고 싶습니다.  
따라서 먼저 “지금 무슨 주제를 말하고 있는지”를 알아야 전환 점수(transition score)를 줄 수 있습니다.


In [4]:
# ============================================================
# 개선된 TopicDetector: 가중치 기반 토픽 탐지
# ============================================================

class ImprovedTopicDetector:
    """
    가중치 기반 토픽 탐지기

    개선 사항:
    1. 키워드별 가중치 적용 (중요 키워드 = 높은 점수)
    2. 여러 키워드 매칭 시 시너지 보너스
    3. 정규화된 점수 (0~1)
    """

    def __init__(self, taxonomy: Dict = TOPIC_TAXONOMY):
        self.taxonomy = taxonomy
        self._build_keyword_index()

    def _build_keyword_index(self):
        """
        키워드 → 토픽 역인덱스 구축
        """
        self.keyword_to_topics = defaultdict(list)

        for topic_id, topic_data in self.taxonomy.items():
            weights = topic_data.get("weight", {})

            for keyword in topic_data["keywords"]:
                # 키워드별 가중치 (기본값 1.0)
                weight = weights.get(keyword, 1.0)
                self.keyword_to_topics[keyword].append((topic_id, weight))

    def detect(self, text: str) -> Dict[str, float]:
        """
        텍스트에서 토픽 점수 계산

        Parameters
        ----------
        text : str
            분석할 텍스트

        Returns
        -------
        Dict[str, float]
            {토픽ID: 점수}
        """
        scores = defaultdict(float)
        matched_keywords = defaultdict(list)

        # 모든 키워드에 대해 검사
        for keyword, topic_weights in self.keyword_to_topics.items():
            if keyword in text:
                for topic_id, weight in topic_weights:
                    scores[topic_id] += weight
                    matched_keywords[topic_id].append(keyword)

        # 시너지 보너스: 여러 키워드 매칭 시 추가 점수
        for topic_id in scores:
            num_keywords = len(matched_keywords[topic_id])
            if num_keywords >= 3:
                scores[topic_id] *= 1.3  # 30% 보너스
            elif num_keywords >= 2:
                scores[topic_id] *= 1.1  # 10% 보너스

        return dict(scores)

    def get_top_topics(
        self,
        text: str,
        top_k: int = 3,
        min_score: float = 0.5
    ) -> List[Tuple[str, float, List[str]]]:
        """
        상위 K개 토픽 반환 (매칭된 키워드 포함)

        Returns
        -------
        List[Tuple[str, float, List[str]]]
            [(토픽ID, 점수, [매칭키워드들]), ...]
        """
        scores = self.detect(text)

        # 매칭된 키워드 찾기
        results = []
        for topic_id, score in scores.items():
            if score >= min_score:
                matched = [
                    kw for kw in self.taxonomy[topic_id]["keywords"]
                    if kw in text
                ]
                results.append((topic_id, score, matched))

        # 점수 내림차순 정렬
        results.sort(key=lambda x: -x[1])

        return results[:top_k]

    def get_primary_topic(self, text: str) -> Optional[str]:
        """가장 높은 점수의 토픽 반환"""
        results = self.get_top_topics(text, top_k=1, min_score=0.0)
        return results[0][0] if results else None


# 테스트
detector = ImprovedTopicDetector()

print(" ImprovedTopicDetector 정의 완료")
print("\n" + "="*60)
print("테스트 결과")
print("="*60)

test_cases = [
    "요즘 팀플 때문에 바빠요. 발표 준비도 해야 하고.",
    "주말에 강아지랑 산책했는데 날씨가 너무 좋았어.",
    "어제 넷플릭스에서 영화 봤는데 진짜 재밌더라.",
    "요즘 회사 야근이 너무 많아서 힘들어.",
    "제주도 여행 가고 싶다. 비행기 표 알아봐야지.",
]

for text in test_cases:
    results = detector.get_top_topics(text, top_k=2)
    print(f"\n \"{text}\"")
    for topic, score, keywords in results:
        topic_name = TOPIC_TAXONOMY[topic]['name_ko']
        print(f"   → {topic_name}: {score:.1f}점 (키워드: {', '.join(keywords)})")

 ImprovedTopicDetector 정의 완료

테스트 결과

 "요즘 팀플 때문에 바빠요. 발표 준비도 해야 하고."
   → 계절/날씨: 1.5점 (키워드: 비)
   → 스포츠/레저: 1.0점 (키워드: 팀)

 "주말에 강아지랑 산책했는데 날씨가 너무 좋았어."
   → 반려동물: 5.0점 (키워드: 강아지, 산책)
   → 계절/날씨: 3.0점 (키워드: 날씨)

 "어제 넷플릭스에서 영화 봤는데 진짜 재밌더라."
   → 영화/만화: 5.5점 (키워드: 영화, 넷플릭스)

 "요즘 회사 야근이 너무 많아서 힘들어."
   → 회사/아르바이트: 3.9점 (키워드: 회사, 야근)

 "제주도 여행 가고 싶다. 비행기 표 알아봐야지."
   → 여행: 7.8점 (키워드: 여행, 제주도, 비행기)
   → 계절/날씨: 1.5점 (키워드: 비)


---
## 4. 관계 분석기

### 4-1. 관계 분석기(RelationshipAnalyzer) 상세 설명

이 섹션은 “누가 누구에게 말하는가?”를 분석해, 추천 토픽에 **관계 적합성 점수**를 부여합니다.

#### 입력/출력
- 입력: `user: PersonProfile`, `avatar: PersonProfile`
- 출력: `relationship: Dict`  
  예) `relationship_type`, `status_gap`, `closeness`, `recommended_formality`, `topic_preferences` 등

#### 관계를 왜 점수화하는가?
같은 주제라도 **관계에 따라 어울림이 다릅니다.**
- 교수 ↔ 학생: 과제/학업/진로는 자연스럽지만, 사적인 연애/사회이슈는 조심
- 친구 ↔ 친구: 취미/연예/여행이 자연스러움
- 낯선 사람: 가벼운 날씨/취미/음식이 안전

#### 관계 분석 로직(개념)
1) **Role 기반 지위 레벨**을 숫자로 매핑 (예: PROFESSOR > SENIOR > FRIEND)
2) 사용자–아바타의 `status_gap` 계산  
3) `status_gap`와 role 조합으로 `relationship_type`을 결정  
4) 관계 타입에 따라:
   - 권장 말투(`recommended_formality`) 결정
   - 토픽 선호도(`topic_preferences`) 테이블을 적용 (각 토픽 가중치)

#### Layer 2와의 연결
Layer 2에서는 “관계 선호도 점수 × (토픽 기본 점수)” 형태로 결합해  
**관계에 맞지 않는 토픽이 상위로 올라오지 않도록** 합니다.


In [5]:
# ============================================================
# 역할 및 관계 정의
# ============================================================

class Role(Enum):
    STUDENT = "student"
    PEER = "peer"
    SENIOR = "senior"
    PROFESSOR = "professor"
    STRANGER = "stranger"

class RelationshipType(Enum):
    FORMAL_HIERARCHICAL = "formal_hierarchical"  # 교수-학생
    SEMI_FORMAL = "semi_formal"                  # 선배-후배
    CASUAL_EQUAL = "casual_equal"                # 친구
    FIRST_MEETING = "first_meeting"              # 처음 만남

ROLE_HIERARCHY = {
    Role.STUDENT: 0,
    Role.PEER: 1,
    Role.SENIOR: 2,
    Role.PROFESSOR: 3,
    Role.STRANGER: 1
}

@dataclass
class PersonProfile:
    name: str
    age: int
    role: Role
    interests: List[str] = field(default_factory=list)
    avoid_topics: List[str] = field(default_factory=list)
    personality: str = ""
    korean_level: str = "native"

print("역할 및 프로필 정의 완료")

역할 및 프로필 정의 완료


In [6]:
# ============================================================
# 관계-토픽 적합성 매트릭스 (20개 토픽 버전)
# ============================================================

TOPIC_COMPATIBILITY = {
    RelationshipType.FORMAL_HIERARCHICAL: {  # 교수-학생
        "가족": 0.3, "연애/결혼": 0.1, "회사/아르바이트": 0.4,
        "교육": 0.95, "군대": 0.3, "주거와생활": 0.3,
        "교통": 0.4, "건강": 0.4, "미용": 0.1,
        "계절/날씨": 0.6, "식음료": 0.4, "상거래전반": 0.2,
        "게임": 0.1, "스포츠/레저": 0.3, "방송/연예": 0.2,
        "영화/만화": 0.3, "여행": 0.5, "반려동물": 0.3,
        "사회이슈": 0.2, "타국가이슈": 0.2
    },
    RelationshipType.SEMI_FORMAL: {  # 선배-후배
        "가족": 0.5, "연애/결혼": 0.4, "회사/아르바이트": 0.8,
        "교육": 0.85, "군대": 0.7, "주거와생활": 0.6,
        "교통": 0.6, "건강": 0.6, "미용": 0.5,
        "계절/날씨": 0.7, "식음료": 0.8, "상거래전반": 0.6,
        "게임": 0.5, "스포츠/레저": 0.7, "방송/연예": 0.6,
        "영화/만화": 0.7, "여행": 0.75, "반려동물": 0.6,
        "사회이슈": 0.3, "타국가이슈": 0.3
    },
    RelationshipType.CASUAL_EQUAL: {  # 친구
        "가족": 0.7, "연애/결혼": 0.8, "회사/아르바이트": 0.8,
        "교육": 0.75, "군대": 0.8, "주거와생활": 0.8,
        "교통": 0.7, "건강": 0.8, "미용": 0.8,
        "계절/날씨": 0.7, "식음료": 0.95, "상거래전반": 0.8,
        "게임": 0.9, "스포츠/레저": 0.9, "방송/연예": 0.9,
        "영화/만화": 0.9, "여행": 0.9, "반려동물": 0.85,
        "사회이슈": 0.4, "타국가이슈": 0.4
    },
    RelationshipType.FIRST_MEETING: {  # 처음 만남
        "가족": 0.2, "연애/결혼": 0.1, "회사/아르바이트": 0.4,
        "교육": 0.5, "군대": 0.3, "주거와생활": 0.4,
        "교통": 0.5, "건강": 0.3, "미용": 0.3,
        "계절/날씨": 0.95, "식음료": 0.7, "상거래전반": 0.5,
        "게임": 0.4, "스포츠/레저": 0.6, "방송/연예": 0.6,
        "영화/만화": 0.7, "여행": 0.8, "반려동물": 0.6,
        "사회이슈": 0.1, "타국가이슈": 0.1
    }
}

print("관계-토픽 적합성 매트릭스 정의 완료")

관계-토픽 적합성 매트릭스 정의 완료


In [18]:
# ============================================================
# RelationshipAnalyzer
# ============================================================

class RelationshipAnalyzer:

    def analyze(self, user: PersonProfile, avatar: PersonProfile) -> Dict:
        user_level = ROLE_HIERARCHY.get(user.role, 1)
        avatar_level = ROLE_HIERARCHY.get(avatar.role, 1)
        status_gap = avatar_level - user_level

        rel_type = self._determine_relationship_type(avatar, status_gap)
        formality = self._determine_formality(rel_type)

        common_interests = set(user.interests) & set(avatar.interests)
        avoid_topics = set(user.avoid_topics) | set(avatar.avoid_topics) | set(SENSITIVE_TOPICS)

        return {
            "user_name": user.name,
            "avatar_name": avatar.name,
            "role_pair": f"{user.role.value}-{avatar.role.value}",
            "status_gap": status_gap,
            "relationship_type": rel_type,
            "recommended_formality": formality,
            "common_interests": list(common_interests),
            "avoid_topics": list(avoid_topics)
        }

    def _determine_relationship_type(self, avatar: PersonProfile, status_gap: int) -> RelationshipType:
        if avatar.role == Role.STRANGER:
            return RelationshipType.FIRST_MEETING
        if avatar.role == Role.PROFESSOR:
            return RelationshipType.FORMAL_HIERARCHICAL
        if avatar.role == Role.SENIOR or status_gap >= 1:
            return RelationshipType.SEMI_FORMAL
        return RelationshipType.CASUAL_EQUAL

    def _determine_formality(self, rel_type: RelationshipType) -> str:
        mapping = {
            RelationshipType.FORMAL_HIERARCHICAL: "very_polite",
            RelationshipType.SEMI_FORMAL: "polite",
            RelationshipType.CASUAL_EQUAL: "informal",
            RelationshipType.FIRST_MEETING: "polite"
        }
        return mapping.get(rel_type, "polite")

print("RelationshipAnalyzer 정의 완료")

RelationshipAnalyzer 정의 완료


---
## 5. 토픽 전환 그래프

### 5-1. 토픽 전환 그래프(Transition Graph) 상세 설명

이 섹션은 “어떤 토픽에서 어떤 토픽으로 넘어가면 자연스러운가?”를 그래프 형태로 모델링합니다.

#### 왜 그래프가 필요한가?
대화는 보통 **연결된 연상 흐름**을 가집니다.
- `식음료 → 여행` (맛집 이야기하다가 여행지로 확장)
- `교육 → 진로` (수업/과제 이야기하다가 커리어로 연결)
- `건강 → 스포츠/레저` (운동/건강 이야기로 자연스럽게 이어짐)

그래프를 두면 Layer 3에서 다음과 같은 처리가 가능합니다.
- **현재 토픽과 가까운 토픽**을 더 위로 올림
- **갑작스러운 전환**(무관한 토픽)을 약간 내려 안정적인 흐름 생성

#### Layer 3 전환 점수(개념)
- 현재 대화 토픽이 `A`일 때 후보 토픽이 `B`라면:
  - 그래프에 `A -> B`가 있으면 +점수
  - 없으면 0 또는 약한 패널티
- 이 점수는 “관계 점수”와 함께 최종 랭킹에 반영됩니다.

#### 확장 포인트
- 실제 앱 로그를 기반으로 “사용자들이 자주 이동하는 토픽 전환”을 통계적으로 추정해
  그래프를 자동 업데이트하는 방식으로 발전시킬 수 있습니다.


In [8]:
# ============================================================
# 토픽 전환 그래프 (자연스러운 대화 흐름)
# ============================================================

TOPIC_TRANSITIONS = {
    "교육": ["회사/아르바이트", "식음료", "스포츠/레저", "계절/날씨"],
    "회사/아르바이트": ["교육", "식음료", "건강", "여행", "주거와생활"],
    "가족": ["여행", "식음료", "건강", "반려동물"],
    "연애/결혼": ["가족", "식음료", "여행", "영화/만화"],
    "건강": ["스포츠/레저", "식음료", "미용"],
    "미용": ["건강", "상거래전반", "방송/연예"],
    "식음료": ["여행", "상거래전반", "방송/연예", "계절/날씨"],
    "여행": ["식음료", "교통", "계절/날씨", "반려동물"],
    "게임": ["영화/만화", "방송/연예", "스포츠/레저"],
    "스포츠/레저": ["건강", "여행", "게임", "계절/날씨"],
    "방송/연예": ["영화/만화", "게임", "식음료"],
    "영화/만화": ["방송/연예", "게임", "식음료"],
    "반려동물": ["가족", "여행", "건강"],
    "주거와생활": ["상거래전반", "회사/아르바이트"],
    "교통": ["여행", "회사/아르바이트", "주거와생활"],
    "상거래전반": ["식음료", "미용", "주거와생활"],
    "계절/날씨": ["여행", "식음료", "스포츠/레저"],  # 스몰토크 → 다른 주제
    "군대": ["가족", "회사/아르바이트", "교육"],
    "사회이슈": [],  # 전환 비추천
    "타국가이슈": []  # 전환 비추천
}

def get_transition_score(from_topic: str, to_topic: str) -> float:
    if from_topic == to_topic:
        return 0.5
    if to_topic in TOPIC_TRANSITIONS.get(from_topic, []):
        return 1.0
    # 간접 연결
    for mid in TOPIC_TRANSITIONS.get(from_topic, []):
        if to_topic in TOPIC_TRANSITIONS.get(mid, []):
            return 0.6
    return 0.3

print(" 토픽 전환 그래프 정의 완료")

 토픽 전환 그래프 정의 완료


---
## 6. 통합 추천 시스템

### 6-1. 통합 추천 시스템(TopicRecommendationSystem) — 레이어별 상세 동작

이 섹션이 노트북의 핵심입니다. 앞의 구성요소들을 조합해 **최종 토픽 추천 + 시작 문장 생성**을 수행합니다.

---

## A) 주요 데이터 구조

### `PersonProfile`
사용자/아바타를 동일한 구조로 표현합니다.
- `role`: 교수/선배/친구/낯선 사람 등
- `interests`: 관심 토픽(개인화)
- `avoid_topics`: 회피 토픽(안전)
- `korean_level`: 언어 수준(문장 난이도/말투 조절용)

### `TopicRecommendation`
추천 결과를 담는 구조체입니다.
- `topic_id`, `topic_name`
- `score`: 최종 점수
- `compatibility_score`: 관계 적합성/맥락 점수 등의 결합 값
- `starter_sentences`: 최종 시작 문장 리스트

---

## B) 레이어별 처리 흐름

### Layer 1: Rule-Based Filtering
목표: **추천하면 안 되는 토픽을 먼저 제거**
- 사용자의 `avoid_topics` 제거
- 관계 타입상 위험한 토픽(예: STRANGER에게 사회이슈 등) 제한
- “안전하고 가벼운 토픽”을 기본 후보로 유지

### Layer 2: Relationship Scoring
목표: “관계에 맞는 토픽을 위로 올리기”
- `RelationshipAnalyzer`가 만든 `topic_preferences` 가중치를 사용
- 예: 교수 관계에서는 “교육/진로/업무” 가중치↑, “연애/사회이슈” 가중치↓

### Layer 3: Context-Aware Ranking
목표: “지금 대화 흐름에서 자연스럽게 이어지는 토픽” 우선
- `ImprovedTopicDetector`로 현재 대화 토픽 후보를 감지
- `TRANSITION_GRAPH`로 전환 가능성이 높은 토픽에 가산점

---

## C) Layer 4: Starter Generation (비용↓, 안정성↑)

이 노트북에서 Layer 4는 다음 원칙을 따릅니다.

### 1) 템플릿 우선(Template-first)
- `STARTER_TEMPLATES[topic_id]`에서 **2개 정도**를 선택
- 장점: 말투/존댓말이 안정적, 데모가 흔들리지 않음

### 2) LLM은 “상위 N개 토픽”만(기본 2개)
- `LLM_TOP_N = 2`
- 상위 토픽 1~2개만 LLM로 더 자연스럽게 다듬는 방식
- 나머지는 템플릿으로 유지 → 비용/지연/불안정성 최소화

### 3) 캐시로 재사용 (핵심)
- 캐시 키: `(topic_id, relationship_type, korean_level)`
- 같은 상황에서 반복 추천될 때 **항상 같은 결과**를 재사용 → 속도↑, 비용↓, 일관성↑

### 4) “반복되면 LLM” 같은 정책도 가능
- `_starter_usage` 같은 카운터로 특정 키가 너무 자주 쓰이면
  LLM 개선을 트리거할 수 있습니다(사용자 경험 개선).

---


In [9]:
# ============================================================
# 대화 시작 문장 템플릿
# ============================================================

STARTER_TEMPLATES = {
    "교육": [
        "요즘 공부 어떻게 하고 있어요?",
        "시험 준비는 잘 되고 있어요?",
        "어떤 과목이 제일 재밌어요?"
    ],
    "회사/아르바이트": [
        "요즘 일은 어때요?",
        "알바 힘들지 않아요?",
        "회사 생활은 어때요?"
    ],
    "식음료": [
        "점심 뭐 먹었어요?",
        "요즘 맛집 다녀온 데 있어요?",
        "커피 좋아해요?"
    ],
    "여행": [
        "최근에 여행 다녀왔어요?",
        "가보고 싶은 곳 있어요?",
        "제주도 가본 적 있어요?"
    ],
    "영화/만화": [
        "요즘 뭐 보고 있어요?",
        "영화 추천해줄 수 있어요?",
        "넷플릭스 뭐 봐요?"
    ],
    "방송/연예": [
        "요즘 보는 드라마 있어요?",
        "좋아하는 가수 있어요?",
        "예능 뭐 봐요?"
    ],
    "게임": [
        "요즘 무슨 게임 해요?",
        "게임 좋아해요?",
        "롤 해봤어요?"
    ],
    "스포츠/레저": [
        "운동 뭐 좋아해요?",
        "주말에 뭐 하고 놀아요?",
        "등산 가본 적 있어요?"
    ],
    "계절/날씨": [
        "오늘 날씨 좋지 않아요?",
        "요즘 많이 춥죠?",
        "비 올 것 같은데요."
    ],
    "반려동물": [
        "강아지 키워요?",
        "반려동물 있어요?",
        "고양이 좋아해요?"
    ],
    "건강": [
        "요즘 운동 하고 있어요?",
        "건강 관리 어떻게 해요?",
        "잠은 잘 자요?"
    ],
    "미용": [
        "머리 어디서 잘라요?",
        "피부 관리 어떻게 해요?",
        "염색 해본 적 있어요?"
    ],
    "주거와생활": [
        "어디 살아요?",
        "자취해요?",
        "집 구하기 힘들지 않아요?"
    ],
    "가족": [
        "가족은 어떻게 지내세요?",
        "형제자매 있어요?",
        "부모님은 어디 사세요?"
    ]
}

print("대화 시작 문장 템플릿 정의 완료")

대화 시작 문장 템플릿 정의 완료


In [10]:
# ============================================================
# TopicRecommendationSystem: 통합 추천 시스템
# ============================================================

@dataclass
class TopicRecommendation:
    topic_id: str
    topic_name: str
    score: float
    compatibility_score: float
    interest_score: float
    transition_score: float
    starter_sentences: List[str]
    reason: str


class TopicRecommendationSystem:
    """
    통합 토픽 추천 시스템

    추천 점수 = 0.40 × 관계적합성 + 0.35 × 관심사 + 0.25 × 전환자연스러움
    """

    W_COMPATIBILITY = 0.40
    W_INTEREST = 0.35
    W_TRANSITION = 0.25

    def __init__(self):
        self.detector = ImprovedTopicDetector()
        self.analyzer = RelationshipAnalyzer()

        # ====================================================
        # Layer 4 (Conversation Starters): cheaper + reliable
        # - templates first
        # - LLM only for top N topics or when templates feel repetitive
        # - cache per (topic + relationship_type + korean_level)
        # ====================================================
        # cache: key -> {"starters": List[str], "is_llm": bool}
        self.starter_cache: Dict[Tuple[str, str, str], Dict] = {}
        self._starter_usage = defaultdict(int)
        self.LLM_TOP_N = 2

    def _rel_key(self, relationship_type) -> str:
        # relationship_type might be Enum
        return getattr(relationship_type, "value", str(relationship_type))

    def _starter_cache_key(self, topic_id: str, relationship: Dict, user: PersonProfile) -> Tuple[str, str, str]:
        return (topic_id, self._rel_key(relationship.get("relationship_type")), user.korean_level)

    def _apply_formality_prefix(self, s: str, relationship: Dict, avatar: PersonProfile) -> str:
        formality = relationship.get("recommended_formality", "polite")

        # professor / very polite cases: add respectful address cue
        if avatar.role == Role.PROFESSOR or formality in ["very_polite"]:
            if not re.match(r"^(교수님|선생님|.*님)[, ]", s):
                return f"{avatar.name}님, {s}"
        return s

    def _template_starters(self, topic_id: str, relationship: Dict, avatar: PersonProfile, k: int = 2) -> List[str]:
        base = STARTER_TEMPLATES.get(topic_id, ["이 주제에 대해 이야기해요."])
        base = base[:]  # copy
        random.shuffle(base)
        starters = [self._apply_formality_prefix(x, relationship, avatar) for x in base[:k]]
        return starters

    def _templates_feel_repetitive(self, key: Tuple[str, str, str]) -> bool:
        # After a couple uses, treat templates as repetitive
        return self._starter_usage[key] >= 2

    def _llm_enhance_starters(
        self,
        topic_name: str,
        base_starters: List[str],
        relationship: Dict,
        user: PersonProfile,
        avatar: PersonProfile,
        current_dialogue: str = ""
    ) -> List[str]:
        '''
        Optional: replace this stub with your CLOVA Studio / OpenAI-compatible call.
        For demo stability, this returns base_starters unless you later wire an API client.
        '''
        if not USE_LLM:
            return base_starters
        return base_starters

    def _get_starters(
        self,
        topic_id: str,
        topic_name: str,
        relationship: Dict,
        user: PersonProfile,
        avatar: PersonProfile,
        current_dialogue: str = "",
        allow_llm: bool = False
    ) -> List[str]:
        key = self._starter_cache_key(topic_id, relationship, user)

        # Count usage (used to detect repetition)
        self._starter_usage[key] += 1

        cached = self.starter_cache.get(key)
        repetitive = self._templates_feel_repetitive(key)

        # If cached exists and we don't need to upgrade to LLM, return
        if cached and not ((allow_llm or repetitive) and not cached.get("is_llm", False)):
            return cached["starters"]

        # templates first
        base = self._template_starters(topic_id, relationship, avatar, k=2)

        # LLM only when allowed (top N) OR templates feel repetitive
        starters = base
        used_llm = False
        if (allow_llm or repetitive) and USE_LLM:
            starters = self._llm_enhance_starters(
                topic_name=topic_name,
                base_starters=base,
                relationship=relationship,
                user=user,
                avatar=avatar,
                current_dialogue=current_dialogue or ""
            )
            used_llm = True

        # cache final
        self.starter_cache[key] = {"starters": starters, "is_llm": used_llm}
        return starters


    def recommend(
        self,
        user: PersonProfile,
        avatar: PersonProfile,
        current_dialogue: str = "",
        current_topic: Optional[str] = None,
        top_k: int = 5
    ) -> Dict:
        """
        토픽 추천 수행
        """
        # 1. 관계 분석
        relationship = self.analyzer.analyze(user, avatar)
        rel_type = relationship["relationship_type"]
        avoid_topics = set(relationship["avoid_topics"])

        # 2. 현재 토픽 탐지
        current_topics = []
        if current_dialogue:
            detected = self.detector.get_top_topics(current_dialogue, top_k=2)
            current_topics = [
                {"id": t[0], "name": TOPIC_TAXONOMY[t[0]]["name_ko"],
                 "score": t[1], "keywords": t[2]}
                for t in detected
            ]
            if detected and not current_topic:
                current_topic = detected[0][0]

        # 3. 각 토픽 점수 계산
        recommendations = []

        for topic_id, topic_data in TOPIC_TAXONOMY.items():
            # 필터링
            if topic_id in avoid_topics:
                continue
            if topic_data.get("is_sensitive", False):
                continue

            # 점수 계산
            compatibility = TOPIC_COMPATIBILITY.get(rel_type, {}).get(topic_id, 0.5)

            interest = 0.5
            if topic_id in user.interests and topic_id in avatar.interests:
                interest = 1.0
            elif topic_id in user.interests or topic_id in avatar.interests:
                interest = 0.7

            transition = 0.5
            if current_topic:
                transition = get_transition_score(current_topic, topic_id)

            final_score = (
                self.W_COMPATIBILITY * compatibility +
                self.W_INTEREST * interest +
                self.W_TRANSITION * transition
            )
            # 시작 문장 (Layer 4에서 top-K에 대해 생성/캐시)
            starters = []

            # 추천 이유
            reasons = []
            if compatibility >= 0.8:
                reasons.append(f"{avatar.name}님과의 관계에 적합")
            if interest >= 0.9:
                reasons.append("공통 관심사")
            elif interest >= 0.7:
                reasons.append("관심 있는 주제")
            if transition >= 0.8:
                reasons.append("자연스러운 전환")
            if not reasons:
                reasons.append("안전한 대화 주제")

            recommendations.append(TopicRecommendation(
                topic_id=topic_id,
                topic_name=topic_data["name_ko"],
                score=round(final_score, 3),
                compatibility_score=round(compatibility, 3),
                interest_score=round(interest, 3),
                transition_score=round(transition, 3),
                starter_sentences=[],
                reason=", ".join(reasons)
            ))

        # 정렬
        recommendations.sort(key=lambda x: -x.score)
        top_recommendations = recommendations[:top_k]

        #  Layer 4: starters (template-first, LLM only for top N, cached)
        for i, r in enumerate(top_recommendations):
            r.starter_sentences = self._get_starters(
                topic_id=r.topic_id,
                topic_name=r.topic_name,
                relationship=relationship,
                user=user,
                avatar=avatar,
                current_dialogue=current_dialogue or "",
                allow_llm=(USE_LLM and i < self.LLM_TOP_N)
            )


        # 결과
        return {
            "relationship": relationship,
            "current_topics": current_topics,
            "recommendations": [
                {
                    "rank": i + 1,
                    "topic_id": r.topic_id,
                    "topic_name": r.topic_name,
                    "score": r.score,
                    "scores": {
                        "compatibility": r.compatibility_score,
                        "interest": r.interest_score,
                        "transition": r.transition_score
                    },
                    "starters": r.starter_sentences,
                    "reason": r.reason
                }
                for i, r in enumerate(top_recommendations)
            ],
            "summary": self._generate_summary(relationship, top_recommendations)
        }

    def _generate_summary(self, relationship: Dict, recommendations: List) -> str:
        if not recommendations:
            return "추천할 토픽이 없습니다."
        top = recommendations[0]
        formality = {"very_polite": "습니다체", "polite": "해요체", "informal": "반말"}
        return (
            f"{relationship['avatar_name']}님과의 대화에서는 "
            f"'{top.topic_name}' 주제를 추천합니다. "
            f"{formality.get(relationship['recommended_formality'], '해요체')}를 사용하세요."
        )

print(" TopicRecommendationSystem 정의 완료")

 TopicRecommendationSystem 정의 완료


---
## 7. 결과 출력 함수

### 7-1. 결과 출력 함수(print_result) 설명

`print_result(result)`는 추천 엔진이 반환한 딕셔너리를 **데모/디버깅용으로 보기 좋게 출력**합니다.

####  출력 내용
- 관계 분석 결과(관계 타입, 지위 차이, 권장 말투)
- 추천 토픽 리스트(점수, 적합성 점수)
- 각 토픽의 시작 문장(템플릿/LLM 결과)

#### 실무적으로 유용한 이유
- 모델이 “왜 이 토픽을 추천했는지”를 빠르게 확인할 수 있음
- 데모 발표에서 시각적으로 결과를 설명하기 쉬움


In [11]:
def print_result(result: Dict):
    rel = result["relationship"]

    print("=" * 65)
    print(" 관계 분석")
    print("=" * 65)
    print(f"   사용자: {rel['user_name']}")
    print(f"   아바타: {rel['avatar_name']}")
    print(f"   관계: {rel['role_pair']}")
    print(f"   권장 말투: {rel['recommended_formality']}")

    if result["current_topics"]:
        print("\n 현재 대화 토픽:")
        for t in result["current_topics"]:
            print(f"    - {t['name']}: {t['score']:.1f}점 (키워드: {', '.join(t['keywords'][:3])})")

    print("\n" + "=" * 65)
    print(" 토픽 추천")
    print("=" * 65)

    for rec in result["recommendations"]:
        print(f"\n  [{rec['rank']}위] {rec['topic_name']}")
        print(f"       점수: {rec['score']:.3f}")
        print(f"       관계:{rec['scores']['compatibility']:.2f} | 관심:{rec['scores']['interest']:.2f} | 전환:{rec['scores']['transition']:.2f}")
        print(f"       이유: {rec['reason']}")
        print(f"       시작: \"{rec['starters'][0]}\"")

    print("\n" + "-" * 65)
    print(f" {result['summary']}")
    print("-" * 65)

print(" 출력 함수 정의 완료")

 출력 함수 정의 완료


---
## 8. 테스트

### 8-1. 테스트(프로필 생성 및 실행) 설명

여기서는 실제 앱 상황을 가정해 `PersonProfile`을 만들고 추천 시스템을 실행합니다.

#### 테스트 시나리오 구성
- 사용자: 나이/관심사/회피 토픽/한국어 수준 설정
- 아바타: 교수/선배/친구 등 Role을 바꿔가며 결과가 어떻게 달라지는지 확인 가능
- `current_dialogue`: 최근 대화 텍스트를 넣어 Layer 3 효과(맥락 반영)를 확인

#### 확인해야 할 포인트(체크리스트)
- 회피 토픽이 실제로 추천에서 빠지는가? (Layer 1)
- 교수/친구/낯선 사람에 따라 추천 토픽이 바뀌는가? (Layer 2)
- 현재 대화 토픽과 자연스럽게 이어지는 주제가 올라오는가? (Layer 3)
- 시작 문장이 템플릿 기반으로 안정적으로 생성되는가? (Layer 4)
- 동일 상황에서 반복 실행 시 결과가 캐시로 재사용되는가? (Layer 4 캐시)


In [12]:
# 프로필 생성
user = PersonProfile(
    name="지민",
    age=23,
    role=Role.STUDENT,
    interests=["게임", "영화/만화", "식음료", "여행"],
    avoid_topics=["사회이슈"],
    korean_level="intermediate"
)

professor = PersonProfile(
    name="김 교수님",
    age=55,
    role=Role.PROFESSOR,
    interests=["교육", "여행", "건강"]
)

senior = PersonProfile(
    name="민수 선배",
    age=26,
    role=Role.SENIOR,
    interests=["게임", "스포츠/레저", "식음료"]
)

friend = PersonProfile(
    name="수진",
    age=22,
    role=Role.PEER,
    interests=["영화/만화", "방송/연예", "식음료", "여행"]
)

system = TopicRecommendationSystem()
print(" 테스트 프로필 생성 완료")

 테스트 프로필 생성 완료


In [13]:
print("\n" + "#" * 65)
print("# 시나리오 1: 교수님과 대화 시작")
print("#" * 65)

result1 = system.recommend(user, professor, top_k=5)
print_result(result1)


#################################################################
# 시나리오 1: 교수님과 대화 시작
#################################################################
 관계 분석
   사용자: 지민
   아바타: 김 교수님
   관계: student-professor
   권장 말투: very_polite

 토픽 추천

  [1위] 교육
       점수: 0.750
       관계:0.95 | 관심:0.70 | 전환:0.50
       이유: 김 교수님님과의 관계에 적합, 관심 있는 주제
       시작: "김 교수님님, 시험 준비는 잘 되고 있어요?"

  [2위] 여행
       점수: 0.675
       관계:0.50 | 관심:1.00 | 전환:0.50
       이유: 공통 관심사
       시작: "김 교수님님, 최근에 여행 다녀왔어요?"

  [3위] 계절/날씨
       점수: 0.540
       관계:0.60 | 관심:0.50 | 전환:0.50
       이유: 안전한 대화 주제
       시작: "김 교수님님, 요즘 많이 춥죠?"

  [4위] 건강
       점수: 0.530
       관계:0.40 | 관심:0.70 | 전환:0.50
       이유: 관심 있는 주제
       시작: "김 교수님님, 잠은 잘 자요?"

  [5위] 식음료
       점수: 0.530
       관계:0.40 | 관심:0.70 | 전환:0.50
       이유: 관심 있는 주제
       시작: "김 교수님님, 점심 뭐 먹었어요?"

-----------------------------------------------------------------
 김 교수님님과의 대화에서는 '교육' 주제를 추천합니다. 습니다체를 사용하세요.
----------------------------------------------

In [14]:
print("\n" + "#" * 65)
print("# 시나리오 2: 선배와 대화 중 (현재: 게임 얘기)")
print("#" * 65)

dialogue = "선배, 요즘 롤 많이 해요? 저 어제 랭크 올렸어요!"
print(f"\n현재 대화: \"{dialogue}\"\n")

result2 = system.recommend(user, senior, current_dialogue=dialogue, top_k=5)
print_result(result2)


#################################################################
# 시나리오 2: 선배와 대화 중 (현재: 게임 얘기)
#################################################################

현재 대화: "선배, 요즘 롤 많이 해요? 저 어제 랭크 올렸어요!"

 관계 분석
   사용자: 지민
   아바타: 민수 선배
   관계: student-senior
   권장 말투: polite

 현재 대화 토픽:
    - 게임: 2.0점 (키워드: 롤)

 토픽 추천

  [1위] 식음료
       점수: 0.820
       관계:0.80 | 관심:1.00 | 전환:0.60
       이유: 민수 선배님과의 관계에 적합, 공통 관심사
       시작: "커피 좋아해요?"

  [2위] 스포츠/레저
       점수: 0.775
       관계:0.70 | 관심:0.70 | 전환:1.00
       이유: 관심 있는 주제, 자연스러운 전환
       시작: "운동 뭐 좋아해요?"

  [3위] 영화/만화
       점수: 0.775
       관계:0.70 | 관심:0.70 | 전환:1.00
       이유: 관심 있는 주제, 자연스러운 전환
       시작: "영화 추천해줄 수 있어요?"

  [4위] 여행
       점수: 0.695
       관계:0.75 | 관심:0.70 | 전환:0.60
       이유: 관심 있는 주제
       시작: "최근에 여행 다녀왔어요?"

  [5위] 게임
       점수: 0.675
       관계:0.50 | 관심:1.00 | 전환:0.50
       이유: 공통 관심사
       시작: "롤 해봤어요?"

-----------------------------------------------------------------
 민수 선배님과의 대화에서는 '식음료' 주제를 추천합니다. 해요

In [15]:
print("\n" + "#" * 65)
print("# 시나리오 3: 친구와 대화 시작")
print("#" * 65)

result3 = system.recommend(user, friend, top_k=5)
print_result(result3)


#################################################################
# 시나리오 3: 친구와 대화 시작
#################################################################
 관계 분석
   사용자: 지민
   아바타: 수진
   관계: student-peer
   권장 말투: polite

 토픽 추천

  [1위] 식음료
       점수: 0.795
       관계:0.80 | 관심:1.00 | 전환:0.50
       이유: 수진님과의 관계에 적합, 공통 관심사
       시작: "커피 좋아해요?"

  [2위] 여행
       점수: 0.775
       관계:0.75 | 관심:1.00 | 전환:0.50
       이유: 공통 관심사
       시작: "제주도 가본 적 있어요?"

  [3위] 영화/만화
       점수: 0.755
       관계:0.70 | 관심:1.00 | 전환:0.50
       이유: 공통 관심사
       시작: "넷플릭스 뭐 봐요?"

  [4위] 교육
       점수: 0.640
       관계:0.85 | 관심:0.50 | 전환:0.50
       이유: 수진님과의 관계에 적합
       시작: "요즘 공부 어떻게 하고 있어요?"

  [5위] 회사/아르바이트
       점수: 0.620
       관계:0.80 | 관심:0.50 | 전환:0.50
       이유: 수진님과의 관계에 적합
       시작: "요즘 일은 어때요?"

-----------------------------------------------------------------
 수진님과의 대화에서는 '식음료' 주제를 추천합니다. 해요체를 사용하세요.
-----------------------------------------------------------------


In [16]:
print("\n" + "#" * 65)
print("# 시나리오 4: 친구와 대화 중 (현재: 맛집 얘기)")
print("#" * 65)

dialogue2 = "어제 홍대 맛집 갔는데 치킨이 진짜 맛있더라! 배달도 되던데."
print(f"\n현재 대화: \"{dialogue2}\"\n")

result4 = system.recommend(user, friend, current_dialogue=dialogue2, top_k=5)
print_result(result4)


#################################################################
# 시나리오 4: 친구와 대화 중 (현재: 맛집 얘기)
#################################################################

현재 대화: "어제 홍대 맛집 갔는데 치킨이 진짜 맛있더라! 배달도 되던데."

 관계 분석
   사용자: 지민
   아바타: 수진
   관계: student-peer
   권장 말투: polite

 현재 대화 토픽:
    - 식음료: 6.5점 (키워드: 맛집, 치킨, 배달)

 토픽 추천

  [1위] 여행
       점수: 0.900
       관계:0.75 | 관심:1.00 | 전환:1.00
       이유: 공통 관심사, 자연스러운 전환
       시작: "제주도 가본 적 있어요?"

  [2위] 식음료
       점수: 0.795
       관계:0.80 | 관심:1.00 | 전환:0.50
       이유: 수진님과의 관계에 적합, 공통 관심사
       시작: "커피 좋아해요?"

  [3위] 영화/만화
       점수: 0.780
       관계:0.70 | 관심:1.00 | 전환:0.60
       이유: 공통 관심사
       시작: "넷플릭스 뭐 봐요?"

  [4위] 방송/연예
       점수: 0.735
       관계:0.60 | 관심:0.70 | 전환:1.00
       이유: 관심 있는 주제, 자연스러운 전환
       시작: "예능 뭐 봐요?"

  [5위] 계절/날씨
       점수: 0.705
       관계:0.70 | 관심:0.50 | 전환:1.00
       이유: 자연스러운 전환
       시작: "오늘 날씨 좋지 않아요?"

-----------------------------------------------------------------
 수진님과의 대화에서는 '여행' 주제를 추천합니다.

---
## 9. 시스템 요약

## 9-1. 운영/확장 가이드 (실제 앱에 붙일 때)

이 노트북은 “연구·보고서·데모”에 최적화되어 있지만, 앱에 붙일 때도 구조가 그대로 확장 가능합니다.

### 운영 시 권장 설정
- 기본은 **템플릿 중심**으로 운영하고, LLM은 상위 1~2개만 사용
- 캐시를 켜서 응답을 빠르게 유지
- 민감 토픽은 프로필/규칙으로 먼저 차단 (LLM 전에)

### 확장 아이디어
1) **대화 로그 기반 전환 그래프 학습**
2) 사용자별 “선호 토픽”을 클릭/체류시간으로 업데이트(온라인 러닝)
3) LLM을 “문장 다듬기(rewrite)”에만 한정해 안전성/일관성 강화
4) 한국어 수준별(초급/중급/고급) 템플릿 분리

### 실수하기 쉬운 포인트
- LLM을 무조건 호출하면: 비용↑, 결과 흔들림↑, 발표 데모 불안정  
- 그래서 이 노트북처럼 “템플릿 우선 + 제한적 LLM + 캐시” 구조가 안전합니다.


In [17]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║     Talkativ 토픽 추천 시스템 (AI-Hub 데이터 기반)               ║
╚══════════════════════════════════════════════════════════════════════╝

 데이터 소스:
┌──────────────────────────────────────────────────────────────────────┐
│ AI-Hub 주제별 텍스트 일상 대화 데이터                                 │
│ - URL: https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=543      │
│ - 10,644개 대화, 20개 토픽                                           │
│ - 실제 대화에서 추출한 키워드 사용                                    │
└──────────────────────────────────────────────────────────────────────┘

 개선된 기능:
┌──────────────────────────────────────────────────────────────────────┐
│ 1. AI-Hub 기반 토픽 분류 (20개 카테고리)                              │
│ 2. 가중치 기반 키워드 매칭 (500+ 키워드)                              │
│ 3. 시너지 보너스 (여러 키워드 매칭 시 추가 점수)                       │
│ 4. 관계-토픽 적합성 매트릭스 (4×20)                                   │
│ 5. 토픽 전환 그래프 (자연스러운 대화 흐름)                             │
└──────────────────────────────────────────────────────────────────────┘

 추천 점수 공식:
┌──────────────────────────────────────────────────────────────────────┐
│ score = 0.40 × compatibility + 0.35 × interest + 0.25 × transition   │
│                                                                      │
│ - compatibility: 관계 유형별 토픽 적합성                              │
│ - interest: 공통 관심사 매칭                                         │
│ - transition: 현재 토픽에서의 자연스러운 전환                          │
└──────────────────────────────────────────────────────────────────────┘
""")


╔══════════════════════════════════════════════════════════════════════╗
║     Talkativ 토픽 추천 시스템 (AI-Hub 데이터 기반)               ║
╚══════════════════════════════════════════════════════════════════════╝

 데이터 소스:
┌──────────────────────────────────────────────────────────────────────┐
│ AI-Hub 주제별 텍스트 일상 대화 데이터                                 │
│ - URL: https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=543      │
│ - 10,644개 대화, 20개 토픽                                           │
│ - 실제 대화에서 추출한 키워드 사용                                    │
└──────────────────────────────────────────────────────────────────────┘

 개선된 기능:
┌──────────────────────────────────────────────────────────────────────┐
│ 1. AI-Hub 기반 토픽 분류 (20개 카테고리)                              │
│ 2. 가중치 기반 키워드 매칭 (500+ 키워드)                              │
│ 3. 시너지 보너스 (여러 키워드 매칭 시 추가 점수)                       │
│ 4. 관계-토픽 적합성 매트릭스 (4×20)                                   │
│ 5. 토픽 전환 그래프 (자연스러운 대화 흐름)                        